In [1]:
import os 
import SimpleITK as sitk
from multiprocessing import Pool, cpu_count
from medpy.io import load, save, get_pixel_spacing
from medpy.filter.smoothing import anisotropic_diffusion

# N4 filtering of nii images

In [2]:
def process_patient(nifti_folder, patient): 
    series = os.listdir(nifti_folder + patient)
    for s in series: 
        img = sitk.ReadImage(nifti_folder + patient + "/" + s + '/image.nii.gz')
        mask_image = sitk.OtsuThreshold(img, 0, 1, 50)
        input_image = sitk.Cast(img, sitk.sitkFloat32)
        corrector = sitk.N4BiasFieldCorrectionImageFilter()
        img_bf_corr = corrector.Execute(input_image, mask_image)
        sitk.WriteImage(img_bf_corr, nifti_folder + patient + "/" + s + '/N4_filtered_image.nii.gz')

In [3]:
nifti_folder = "/home/tachennf/Documents/delta-rad/data/Madrid/nifti_data/"
patients_list = os.listdir(nifti_folder)
patients_list = ['Mont1']
for p in patients_list: 
    process_patient(nifti_folder, p)
    print("Processed patient " + p + " with N4 bias field correction")


Processed patient Mont1 with N4 bias field correction


In [4]:
nifti_folder = "/home/tachennf/Documents/delta-rad/data/Madrid/nifti_data/"
patients_list = os.listdir(nifti_folder)
print("Number of cores: ", cpu_count())
with Pool(cpu_count()) as pool:
    pool.starmap(process_patient, [(nifti_folder, p) for p in patients_list])

Number of cores:  16


## Noise filtering
Using Anisotropic diffusion (Perona-Malik). 
Settings:
> Number of iteration: 5

> kappa: 5

> gamma: 3

In [5]:
def noise_process_patient(nifti_folder, patient, num_it=5, kappa=5, gamma=3):
    series = os.listdir(nifti_folder + patient)
    for s in series: 
        img_array, header = load(nifti_folder + patient + "/" + s + '/N4_filtered_image.nii.gz')
        filtered_img_array = anisotropic_diffusion(img_array, num_it, kappa, gamma, get_pixel_spacing(header))
        filtered_img_array[filtered_img_array<=0] = 0
        save(filtered_img_array, nifti_folder + patient + "/" + s + '/Aniso_N4_filtered_image.nii.gz', header)

In [6]:
print("Noise filtering runs.")
with Pool(cpu_count()) as pool:
    pool.starmap(noise_process_patient, [(nifti_folder, p) for p in patients_list])
print("Noise filtering done.")

Noise filtering runs.
Noise filtering done.
